In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *

In [7]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.memory", "4g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Config

In [8]:
environment = '' # dev;uat;prd

In [ ]:
base_path = '../../../data'  # This will be overwritten by Papermill

In [9]:
path = {
    'slv': {'sellers': f'{base_path}/silver/mtd/sellers'}, 
    'snk': f'{base_path}/gold/dim/dim_sellers'
}

## Load

In [10]:
df = spark.read.format('delta').load(path['slv']['sellers'])

In [11]:
df_snk = df.select(
    F.col('id_seller'),
    F.col('cd_seller_zip_prefix'),
    F.col('nm_seller_city'),
    F.col('sg_seller_state'),
)

In [12]:
df_snk = df_snk.withColumn("dh_insert_gld", F.current_timestamp())

## Sink

In [ ]:
df_snk.write.format('delta')\
.mode(saveMode='overwrite')\
.save(path['snk'])